In [23]:
import os
import cv2
import numpy as np
import subprocess
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrixfrom
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

#os: This is a built-in Python library used for interacting with the operating system. It is used in this code to change the current working directory.
#cv2: This is OpenCV, a popular library for computer vision tasks. It is used here for image processing tasks like reading and resizing images.
#numpy: This is a library for numerical computing in Python. It is used here for various array operations.
#subprocess: This is a built-in Python library used to spawn new processes, connect to their input/output/error pipes, and obtain their return codes. It is used here to run the convert command from the ImageMagick suite to convert images to grayscale.
#sklearn.model_selection: This is a module from scikit-learn, a popular machine learning library in Python. It is used here for splitting the dataset into training and testing sets.
#tensorflow.keras.models: This is a module from the Keras API in TensorFlow, a popular deep learning framework. It is used here for defining the CNN model.
#tensorflow.keras.layers: This is a module from the Keras API in TensorFlow, used for defining the layers of the CNN model.
#tensorflow.keras.utils: This is a module from the Keras API in TensorFlow, used for various utility functions like one-hot encoding of labels.
#sklearn.metrics: This is a module from scikit-learn, used for evaluating the performance of the model.
#sklearn.preprocessing: This is a module from scikit-learn, used for preprocessing the data.
#sklearn.utils: This is a module from scikit-learn, used for various utility functions like shuffling the data.
#Overall, this code imports the necessary libraries and modules to build a CNN model for image classification.

In [24]:
# Step 1: Load and Preprocess Datasets
def load_and_preprocess_dataset(dataset_path):
    images = []
    labels = []

    for label in ["Malignant", "Benign"]:
        label_path = os.path.join(dataset_path, label)
        for filename in os.listdir(label_path):
            img_path = os.path.join(label_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (100, 100))  # Resize to a standard size
            images.append(img)
            labels.append(1 if label == "Malignant" else 0)  # 1 for Malignant, 0 for Benign

    return np.array(images), np.array(labels)

dataset1_path = "C:\\Users\\Smart\\OneDrive\\Desktop\\Grad. Proj\\datasets\\breast_cancer_data\\US"
dataset2_path = "C:\\Users\\Smart\\OneDrive\\Desktop\\Grad. Proj\\datasets\\breast_cancer_data\DDSM Dataset"
dataset3_path = "C:\\Users\\Smart\\OneDrive\\Desktop\\Grad. Proj\\datasets\\breast_cancer_data\\MRI"

dataset1_images, dataset1_labels = load_and_preprocess_dataset(dataset1_path)
dataset2_images, dataset2_labels = load_and_preprocess_dataset(dataset2_path)
dataset3_images, dataset3_labels = load_and_preprocess_dataset(dataset3_path)


In [25]:
# Step 2: Combine and Split Datasets
all_images = np.concatenate([dataset1_images, dataset2_images, dataset3_images])
all_labels = np.concatenate([dataset1_labels, dataset2_labels, dataset3_labels])

X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

In [26]:
# Step 3: Build and Train the CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X_train = X_train.reshape(X_train.shape[0], 100, 100, 1)
X_test = X_test.reshape(X_test.shape[0], 100, 100, 1)

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
677/677 [==============================] - 77s 111ms/step - loss: 0.9801 - accuracy: 0.7615 - val_loss: 0.3990 - val_accuracy: 0.8000
Epoch 2/10
677/677 [==============================] - 74s 109ms/step - loss: 0.3175 - accuracy: 0.8438 - val_loss: 0.3263 - val_accuracy: 0.8491
Epoch 3/10
677/677 [==============================] - 81s 120ms/step - loss: 0.2517 - accuracy: 0.8810 - val_loss: 0.2913 - val_accuracy: 0.8728
Epoch 4/10
677/677 [==============================] - 68s 100ms/step - loss: 0.1952 - accuracy: 0.9143 - val_loss: 0.2874 - val_accuracy: 0.8848
Epoch 5/10
677/677 [==============================] - 67s 98ms/step - loss: 0.1694 - accuracy: 0.9307 - val_loss: 0.2814 - val_accuracy: 0.8940
Epoch 6/10
677/677 [==============================] - 79s 116ms/step - loss: 0.1275 - accuracy: 0.9463 - val_loss: 0.3228 - val_accuracy: 0.8986
Epoch 7/10
677/677 [==============================] - 77s 114ms/step - loss: 0.1069 - accuracy: 0.9578 - val_loss: 0.2861 - val_acc

In [27]:
# Assuming you have predictions from Model 1 and Model 2
predictions_model1 = np.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0])
predictions_model2 = np.array([1, 1, 1, 1, 0, 0, 0, 0, 1, 1])

# Combine predictions using a fusion method (e.g., voting)
fusion_predictions = np.logical_or(predictions_model1, predictions_model2).astype(int)

# Assuming ground truth labels
ground_truth = np.array([1, 0, 1, 1, 0, 1, 0, 1, 1, 0])

# Calculate confusion matrix
confusion_mat = confusion_matrix(ground_truth, fusion_predictions)


In [28]:

# Step 4: User Input and Prediction
def predict_image(model, image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (100, 100))
    img = img.reshape(1, 100, 100, 1)
    prediction = model.predict(img)
    return "Malignant" if prediction[0, 0] > 0.5 else "Benign"

user_input_image_path = "C:/Users/Smart/OneDrive/Desktop/Grad. Proj/datasets/breast_cancer_data/DDSM Dataset/Malignant/20586934 (17).png"
#"C:/Users/Smart/OneDrive/Desktop/Grad. Proj/datasets/breast_cancer_data/DDSM Dataset/Benign/20586908 (2).png"
#"C:\\Users\\Smart\\OneDrive\\Desktop\\Grad. Proj\\datasets\\breast_cancer_data\\DDSM Dataset\\Malignant\\20586934 (40).png"
result = predict_image(model, user_input_image_path)
print(f"The image is predicted as {result}.")
print("Confusion Matrix:")
print(confusion_mat)

1/1 [==============================] - 1s 531ms/step
The image is predicted as Malignant.
Confusion Matrix:
[[2 2]
 [1 5]]
